In [1]:
%%file ClusteringTrainKM.py

import skimage
import os
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
from skimage import data
from skimage import io
import glob
from skimage.viewer import ImageViewer
import cv2
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.externals import joblib

class ClusteringKM:
    
    #Initialization
    def __init__(self, train_path, save_path):
        self.train_path = train_path
        self.save_path = save_path
        
    '''#Helper Function to get images from train path
    def get_train_images(self, user_list):
        list_users = user_list
        self.train_imgs = {}
        for i in list_users:
            temp_path = self.train_path + i
            for j in glob.glob(temp_path + '/*.jpg'):
                file_name = j.replace(temp_path,'')[1:]
                img = io.imread(j)
                self.train_imgs[(i,file_name)] = img 
        print("Number of images loaded:", len(self.train_imgs))'''

    def get_train_images(self, train_path, user_list):
        #list_users = user_list
        self.train_imgs = {}
        print(len(user_list))
        for i in user_list:
            file_name = i
            img = io.imread(train_path+str(i)+'.jpg')
            print(train_path+str(i)+'.jpg')
            self.train_imgs[(i,file_name)] = img 
        print("Number of images loaded:", len(self.train_imgs))

    
    #Helper function to convert image to d-dimension vector for each image and 
    #return dataframe of all images
    def convert_to_features(self, columns):
        features = []
        for i in self.train_imgs.items():
            r_mean, r_std, r_med = np.mean(i[1][:,:,0].ravel()), np.std(i[1][:,:,0].ravel()), np.median(i[1][:,:,0].ravel())
            g_mean, g_std, g_med  = np.mean(i[1][:,:,1].ravel()), np.std(i[1][:,:,1].ravel()), np.median(i[1][:,:,1].ravel())
            b_mean, b_std, b_med  = np.mean(i[1][:,:,2].ravel()), np.std(i[1][:,:,2].ravel()), np.median(i[1][:,:,2].ravel())
            canny = np.mean(np.ravel(cv2.Canny(cv2.cvtColor(i[1], cv2.COLOR_BGR2HSV),100,200,L2gradient = True)))
            try:
                orb = cv2.ORB_create(100)
                kp = orb.detect(i[1],None)
                kp, des = orb.compute(i[1], kp)
                orb_centers = list(KMeans(1).fit([i.pt for i in kp]).cluster_centers_)
                orbx1, orby1 = orb_centers[0][0]*255/np.shape(i[1])[0], orb_centers[0][1]*255/np.shape(i[1])[1]
            except ValueError:
                continue
            features.append(np.array([i[0][0],i[0][1], r_mean, r_std, r_med, g_mean, g_std, g_med, b_mean, b_std, b_med, canny, orbx1, orby1]))
        df = pd.DataFrame(features, columns = columns)
        return df
    
    def model_images_fit(self, df, k, extra_cols, rand_state):
        data = df.copy(deep=True)
        
        #Delete reference columns
        for i in extra_cols:
            del data[i]

        #Implement Gaussian Mixture Model Algortihm 
        model = GaussianMixture(n_components=k, random_state=9001)

        #Fit Model and Predict
        model.fit(data)
        y_pred = model.predict_proba(data)

        #Add prediction to dataframe and return 
        for i in range(0,k+1):
            if i==k:
                label = "Prediction"
                df[label] = model.predict(data)
                break
            label = "Prob_" + str(i)
            df[label] = y_pred[:,i]
        return df, model
    
    def model_users_fit(self, df, k, extra_cols, rand_state):
        data = df.copy(deep=True)
        
        #Delete reference columns
        for i in extra_cols:
            del data[i]

        #Implement K-Means Algortihm
        model = KMeans(n_clusters=k, random_state=rand_state)

        #Fit Model, Predict and Return
        model.fit(data)
        y_pred = model.predict(data)
        df['Prediction'] = model.labels_
        return df, model
    
    #Helper function to create folders for Image Clustering
    def save_clusters(self, df, label):
        self.save_path += label + "/"
        for i in self.train_imgs.items():
            temp_row = df[df["URL"]==i[0][1]]
            try:
                name, pred_folder = i[0][1], str(temp_row['Prediction'].values[0])
            except IndexError:
                continue
            temp_path = self.save_path + "Cluster" + pred_folder + "/"
            if not os.path.exists(temp_path):
                os.makedirs(temp_path)
            io.imsave(temp_path+name, i[1])
        print("All Images Saved.")
            
    #Helper function to obtain percentage of Cluster Presence
    def get_cluster_presence(self, df, k):
        cluster_presence = []
        for i in list(df['User_Handle'].unique()):
            user_dict = {}
            temp_df = df[df['User_Handle']==i]
            post_count  =  len(temp_df)
            user_dict['User_Handle'] = i
            for j in range(0, k):
                user_dict["Cluster_"+str(j)] = sum(temp_df['Prob_'+str(j)])/post_count
            cluster_presence.append(user_dict)
        df_presence = pd.DataFrame(cluster_presence)
        df_presence = df_presence.fillna(0)
        return df_presence
    
    #Helper function to save model 
    def save_model(self, model, path):
        joblib.dump(model, path) 
        print("Model Saved.")


import skimage
import os
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
from skimage import data
from skimage import io
import glob
from skimage.viewer import ImageViewer
import cv2
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.externals import joblib

class Ranking:
    
    #Initialization
    def __init__(self, target_path):
        self.target_path = target_path

    #Helper Function to get images from target path 
    def get_images_target(self):
        #Get all Images of Users in the List
        self.target_imgs = {}
        for j in glob.glob(self.target_path + '/*.jpg'):
            file_name = j.replace(self.target_path,'')[:]
            img = io.imread(j)
            self.target_imgs[("Input/Target",file_name)] = img
        print("Number of images loaded:", len(self.target_imgs))
    
    #Helper function to load saved model
    def load_model(self, path):
        model = joblib.load(path)
        print ("Model loaded.")
        return model

    #Helper function to convert image to d-dimension vector for each image and 
    #return dataframe of all images
    def convert_to_features(self, columns):
        features = []
        for i in self.target_imgs.items():
            r_mean, r_std, r_med = np.mean(i[1][:,:,0].ravel()), np.std(i[1][:,:,0].ravel()), np.median(i[1][:,:,0].ravel())
            g_mean, g_std, g_med  = np.mean(i[1][:,:,1].ravel()), np.std(i[1][:,:,1].ravel()), np.median(i[1][:,:,1].ravel())
            b_mean, b_std, b_med  = np.mean(i[1][:,:,2].ravel()), np.std(i[1][:,:,2].ravel()), np.median(i[1][:,:,2].ravel())
            canny = np.mean(np.ravel(cv2.Canny(cv2.cvtColor(i[1], cv2.COLOR_BGR2HSV),100,200,L2gradient = True)))
            try:
                orb = cv2.ORB_create(100)
                kp = orb.detect(i[1],None)
                kp, des = orb.compute(i[1], kp)
                orb_centers = list(KMeans(1).fit([i.pt for i in kp]).cluster_centers_)
                orbx1, orby1 = orb_centers[0][0]*255/np.shape(i[1])[0], orb_centers[0][1]*255/np.shape(i[1])[1]
            except ValueError:
                continue
            features.append(np.array([i[0][0],i[0][1], r_mean, r_std, r_med, g_mean, g_std, g_med, b_mean, b_std, b_med, canny, orbx1, orby1]))
        df = pd.DataFrame(features, columns = columns)
        return df

    #Helper function to make prediction for target images using image model
    def predict(self, df, model, k, cluster_names, extra_cols):
        data = df.copy(deep=True)
        
        #Delete reference columns
        for i in extra_cols:
            del data[i]
            
        #Make Prediction
        y_pred = model.predict_proba(data)
        
        #Add prediction to dataframe and return 
        for i in range(0,k+1):
            if i==k:
                label = "Prediction"
                df[label] = model.predict(data)
                break
            label = cluster_names[i] + " (" + str(i) + ")"
            df[label] = y_pred[:,i]
        return df
    
    #Helper function to generate distance dictionary
    def get_result(self, df, k, model):
        final_dict = {}
        for i in range(0,len(df)):
            temp_file = df.iloc[i,1]
            temp_dist = []
            for j in range(0, k):
                temp_dist.append(np.linalg.norm(df.iloc[i,14:14+k].astype(float)-model.cluster_centers_[j]))
            final_dict[temp_file] = temp_dist
        return final_dict

Overwriting ClusteringTrainKM.py


In [3]:
import cv2 ## issues 
import skimage
from skimage import io
import pickle
#define File-Path to Users Folder
#import scikit-image
import skimage
import os
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
from skimage import data
from skimage import io
from sklearn.mixture import GaussianMixture
import glob
from skimage.viewer import ImageViewer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
%matplotlib inline

#img = io.imread('~/Users/kimia/Desktop/Capstone/imgcluster/Cluster4/11117159_1591691794419786_1496739027_n.jpg')

/anaconda/envs/python3/lib/python3.6/site-packages/skimage/viewer/__init__.py:6: UserWarning: Viewer requires Qt
  warn('Viewer requires Qt')


In [5]:


import ClusteringTrainKM

train_path = '/Users/kimia/Desktop/Capstone/imgcluster/Cluster4/'
save_path = '/Users/kimia/Desktop/Capstone/imgcluster/Cluster4/'
dest_img =  '/Users/kimia/Desktop/Capstone/imgcluster/Cluster4/'
userlist= [1,2,3]
#user list will be the picture number! 

#Helper Function to get images from train path


c = ClusteringTrainKM.ClusteringKM(train_path, save_path )
c.get_train_images(train_path, user_list = ['1','2','3'])
#Create Object


#obj_train = ClusteringTrain.Clustering(train_path, dest_path)

3
/Users/kimia/Desktop/Capstone/imgcluster/Cluster4/1.jpg
/Users/kimia/Desktop/Capstone/imgcluster/Cluster4/2.jpg
/Users/kimia/Desktop/Capstone/imgcluster/Cluster4/3.jpg
Number of images loaded: 3


In [6]:
#Get train dataframe
train_df_cols = ["User_Handle","URL","R_Mean", "R_STD", "R_MED", "G_Mean", "G_STD", 
                "G_MED", "B_Mean", "B_STD", "B_MED", "Canny", "ORB_X", "ORB_Y"]

train_df = c.convert_to_features(train_df_cols)
display(train_df.head())

,User_Handle,URL,R_Mean,R_STD,R_MED,G_Mean,G_STD,G_MED,B_Mean,B_STD,B_MED,Canny,ORB_X,ORB_Y
0,1,1,135.0129197103678,87.63115693479644,153.0,115.81277500106796,85.89295297982054,109.0,58.67071158528771,67.8223132968387,33.0,22.53199891067538,146.9227747599284,130.7175958633423
1,2,2,182.62494660173437,71.07541423145153,214.0,114.09603945063864,61.1908801925312,116.0,59.164076850783886,49.102368231584244,42.0,15.82652505446623,158.0485753377279,124.39350624084474
2,3,3,224.64943771626298,37.1131291445464,242.0,177.51967726088256,68.5927534247089,184.0,103.62004998077663,101.47531061170602,66.0,22.367919389978212,127.65924498240155,189.51394405364988


In [7]:
# Model

In [9]:
from sklearn.externals import joblib
model = joblib.load('model_images.plk')